In [92]:
import numpy as np 
from numpy.linalg import norm, svd

def inexact_augmented_lagrange_multiplier(X, lmbda = 0.1, tol = 1e-7, maxIter = 1000):
    Y = X
    norm_two = norm(Y.ravel(), 2)
    norm_inf = norm(Y.ravel(), np.inf) / lmbda
    dual_norm = np.max([norm_two, norm_inf])
    Y = Y /dual_norm
    A = np.zeros(Y.shape)
    E = np.zeros(Y.shape)
    dnorm = norm(X, 'fro')
    mu = 1.25 / norm_two
    rho = 1.5
    sv = 10.
    n= Y.shape[1]
    itr = 0
    while True:
        Eraw = X - A + (1/mu) * Y
        Eupdate = np.maximum(Eraw - lmbda / mu, 0) + np.minimum(Eraw + lmbda / mu, 0)
        U, S, V = svd(X - Eupdate + (1 / mu) * Y, full_matrices=False)
        svp = (S > 1 / mu).shape[0]
        if svp < sv:
            sv = np.min([svp + 1, n])
        else:
            sv = np.min([svp + round(0.05 * n), n])

        Aupdate = np.dot(np.dot(U[:, :svp], np.diag(S[:svp] - 1 / mu)), V[:svp, :])
        A = Aupdate
        E = Eupdate
        print (itr)
        Z = X - A - E
        Y = Y + mu * Z
        mu = np.min([mu * rho, mu * 1e7])
        itr += 1
        if ((norm(Z, 'fro') / dnorm) < tol) or (itr >= maxIter):
            break
    print("IALM Finished at iteration %d" % (itr))
    return A, E

In [213]:
import numpy as np
from numpy.linalg import norm, svd

def IALM(D, lmbda = 0.1, tol = 1e-7, maxIter = 1000):
    # initialize
    Y = D
    norm_two = norm(D.ravel(), 2)
    norm_inf = norm(D.ravel(), np.inf) / lmbda
    dual_norm = np.max([norm_two, norm_inf])
    Y = D / dual_norm
    
    A_hat = np.zeros(Y.shape)
    E_hat = np.zeros(Y.shape)
    mu = 1.25 / norm_two
    mu_bar = mu * 1e7
    rho = 1.5
    d_norm = norm(D, 'fro')

    iter = 0 
    total_svd = 0 
    converged = False
    stopCriterion = 1
    sv = 10 
    while (converged == False):         
        iter = iter + 1
        
        temp_T = D - A_hat + (1 / mu) * Y
        E_hat = np.maximum(temp_T - lmbda / mu, 0) + np.minimum(temp_T + lmbda / mu, 0)
        U, S, V = svd(D - E_hat + (1 / mu) * Y, full_matrices=False)
        
        svp = (S > 1 / mu).shape[0]
        if svp < sv:
            sv = np.min([svp + 1, n]) 
        else:
            sv = np.min([svp + round(0.05 * n), n]) 
        
        A_hat = np.dot(np.dot(U[:, :svp], np.diag(S[:svp] - 1 / mu)), V[:svp, :])
        # np.dot(np.dot(U[:, :svp], np.diag(S[:svp] - 1 / mu)), V[:svp, :]) 

        total_svd = total_svd + 1 
        
        Z = D - A_hat - E_hat 
        
        Y = Y + mu * Z 
        mu = np.min([mu * rho, mu_bar]) 

        stopCriterion = norm(Z, 'fro') / d_norm 
        print("Iteration %d: mu = %f, sv = %d, stopCriterion = %f" % (iter, mu, sv, stopCriterion))
        if (stopCriterion < tol) or (iter >= maxIter):
            converged = True 
        else:
            converged = False
    print("IALM Finished at iteration %d" % (iter))
    return A_hat, E_hat

In [214]:

import numpy as np
from scipy.stats import ortho_group

def generate(n=100,s=200,rank=5):
    s = (int)(0.05 * (n**2))
    rank = (int)(0.05 * n)
    m=ortho_group.rvs(n)
    m=m[:rank]
    for i in range(rank):
        m[i]=m[i]*np.random.rand()*100
    low_rank=np.matmul(m.T,m)
    position=np.random.randint(0,n**2,s)
    for i in position:
        low_rank[i//n][i % n]+=(np.random.rand()-0.5)*100
    return low_rank

In [228]:
result = inexact_augmented_lagrange_multiplier(generate(n=1000), lmbda=0.1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
IALM Finished at iteration 38


In [229]:
np.sum(np.abs(result[1])>1e-2)

42850

In [230]:
np.sum(np.abs(result[1]) > 1e-2)

42850

In [231]:
np.linalg.matrix_rank(result[0])

1000

In [78]:
E = inexact_augmented_lagrange_multiplier(generate(n = 500, s = 10000, rank = 100))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
IALM Finished at iteration 36


In [83]:
np.sum(np.abs(E[1]) > 1)

9430

In [58]:
import time
start = time.perf_counter()
D = generate(100, 200, 5)
result = inexact_augmented_lagrange_multiplier(D)
end = time.perf_counter()
print("time consuming: ", end - start)

A = result[0]
E = result[1]
np.sum(np.abs(E)<10)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
IALM Finished at iteration 44
time consuming:  0.26505900000120164


3452

In [43]:
A

array([[ 477.19211543, -256.57598896,  -97.18881889, ...,    7.69830278,
          75.63782441,  113.8659243 ],
       [-264.64404402,  638.53637748, -149.50657516, ...,  227.13671337,
        -165.70528904,  -24.39064209],
       [-107.87706039, -152.0400572 ,  312.84791291, ..., -124.27455345,
        -123.73281255, -153.68937529],
       ...,
       [  14.40060734,  230.26542373, -110.67683313, ...,  580.90046868,
         132.37244874,  -83.37615985],
       [  72.78153277, -165.89505768, -113.02342772, ...,  140.16660185,
         432.72850937,  141.27290589],
       [ 120.06679813,  -21.68386349, -148.59862322, ...,  -86.67361693,
         144.53693221,  351.60761221]])

In [56]:
E

array([[32.47105474,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [31]:
E

array([[ 1.67142080e+00,  7.41955669e-02, -3.07260983e+00, ...,
         5.05286600e-01, -1.43612351e-02, -1.30532179e+00],
       [ 4.87851514e-01, -1.50242862e+00,  5.82356681e+00, ...,
        -4.98195708e-01, -1.87400325e-03, -1.07820836e+00],
       [ 5.66260829e-01,  3.39747318e+00,  8.68440303e+01, ...,
        -1.20573237e+01, -3.98225572e-01, -2.52936591e+01],
       ...,
       [ 4.38783406e-01, -3.82624769e-01, -1.42835420e+01, ...,
         2.30436878e-01, -8.47149179e-01, -3.34534827e-01],
       [ 2.63527232e-02, -1.91175685e-01, -6.57861246e-01, ...,
        -2.81567472e-01,  2.65437622e+00,  1.62417048e+00],
       [-2.47005989e+00, -1.31084846e+00, -2.49740970e+01, ...,
        -7.66510704e-02,  1.11803466e+00,  1.94092164e+01]])

In [45]:
A+E

array([[ 549.62634371, -280.02343854, -113.95968383, ...,   -1.01287557,
          65.03410359,  153.91273941],
       [-280.02341649,  745.32096765, -192.87048182, ...,  318.24031493,
        -165.42720055,  -13.32042791],
       [-113.95967118, -192.87050169,  348.51386951, ..., -161.15162498,
        -130.30252057, -168.27869747],
       ...,
       [  -1.01289757,  318.2403256 , -161.15159123, ...,  965.75186815,
         198.76921222, -110.05095035],
       [  65.03408512, -165.42716662, -130.30252533, ...,  198.76920089,
         481.51216212,  141.4330283 ],
       [ 153.91276387,  -13.32043942, -168.27869335, ..., -110.05096751,
         141.43302639,  419.91951154]])

In [46]:
D

array([[ 549.62636486, -280.0234341 , -113.95968631, ...,   -1.0128952 ,
          65.03410092,  153.91274898],
       [-280.0234341 ,  745.32097596, -192.87050383, ...,  318.24030931,
        -165.42719167,  -13.32043424],
       [-113.95968631, -192.87050383,  348.51387545, ..., -161.15160828,
        -130.30253588, -168.27869561],
       ...,
       [  -1.0128952 ,  318.24030931, -161.15160828, ...,  965.75188253,
         198.76918208, -110.05093684],
       [  65.03410092, -165.42719167, -130.30253588, ...,  198.76918208,
         481.51216533,  141.43302657],
       [ 153.91274898,  -13.32043424, -168.27869561, ..., -110.05093684,
         141.43302657,  419.91951588]])

In [234]:
np.linalg.matrix_rank(inexact_augmented_lagrange_multiplier(generate(n = 500, s = 10000, rank = 100))[0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
IALM Finished at iteration 44


500